# MINOR PROJECT EXAMPLES

In [1]:
import os
import sys
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import class_weight
from tqdm import tqdm_notebook as tqdm
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation, Input, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.optimizers import SGD, Adam, RMSprop
from keras.applications import ResNet50
from keras.applications.vgg16 import VGG16
from keras.callbacks import TensorBoard, ModelCheckpoint

Using TensorFlow backend.


In [ ]:
def ResNet50_Model():
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # x = Dense(1000,
    #           kernel_regularizer=regularizers.l2(5e-4),
    #           activation='relu')(x)
    # x = Dropout(0.5)(x)
    predictions = Dense(7, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # model.compile(loss="categorical_crossentropy", 
    #               optimizer=Adam(lr=1e-3), 
    #               metrics=["acc"])
    print(model.summary())

In [ ]:
def VGG16_Model():
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

    x = base_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(7, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

#     for layer in base_model.layers:
#         layer.trainable = False

#     model.compile(loss="categorical_crossentropy", 
#                   optimizer=RMSprop(lr=1e-5), 
#                   metrics=["acc"])
    
    print(model.summary())
    return model

In [ ]:
vgg16_model = VGG16_Model()
resnet50_model = ResNet50_Model()

In [ ]:
VGG16_WEIGHT_FILE = drive_base_path + "models/VGG16_AUG_ISIC_cnn.h5"
RESNET50_WEIGHT_FILE = drive_base_path + "models/checkpoint/ResNet50_REST.14-1.38.hdf5"

vgg16_model.load_weights(VGG16_WEIGHT_FILE)
resnet50_model.load_weights(RESNET50_WEIGHT_FILE)

In [ ]:
def load_sample(path_to_images, number_of_images):
    list_of_images = os.listdir(path_to_images)
    x = []
    for i, filename in enumerate(list_of_images):
        image = cv2.imread(path_to_images + filename)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        x.append(image)
        
        if i >= number_of_images:
            break
    
    x = np.array(x)
    return x       

In [ ]:
datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    rotation_range=30,
    zoom_range=0.2,
    fill_mode='constant',
    cval=0,
    validation_split=0.1,
    data_format="channels_last")

In [ ]:
datagen.fit(x)

In [ ]:
classes = ['mel', 'nv', 'bcc', 'akiec', 'bkl', 'df', 'vasc']
batch_size = 16
train_generator = datagen.flow_from_directory(
                    directory=path_to_balanced_images,
                    target_size=(224, 224),
                    batch_size=batch_size,
                    classes=classes,
                    shuffle=True,
                    subset='training')

validation_generator = datagen.flow_from_directory(
                    directory=path_to_balanced_images,
                    target_size=(224, 224),
                    batch_size=batch_size,
                    classes=classes,
                    shuffle=True,
                    subset='validation')

In [ ]:
Y_pred = model.predict_generator(validation_generator, steps=valid_samples / 1)
y_pred = np.argmax(Y_pred, axis=1)
y_true = validation_generator.classes